In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
import json

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        last_content = ""  # 마지막 content를 저장할 변수

        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    # 데이터 스트림에서 각 라인을 UTF-8로 디코딩
                    decoded_line = line.decode("utf-8")

                    # JSON 형식의 라인에서 'data:'로 시작하는 부분만 추출
                    if decoded_line.startswith('data:'):
                        try:
                            # 'data:' 뒤의 부분을 JSON으로 파싱
                            data_json = decoded_line.split('data:', 1)[1]
                            data_dict = json.loads(data_json)
                            # 'content' 값 추출
                            content = data_dict.get("message", {}).get("content", "")
                            if content:  # content가 비어있지 않은 경우 업데이트
                                last_content = content
                        except json.JSONDecodeError:
                            continue  # JSON 파싱 오류가 발생하면 다음 라인으로
        return last_content

        print("Last content extracted:")
        print(last_content)

In [7]:
if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY6lTkg/kCOEv6QZjvkQU61uo3PQoaid6Aols2EIOSYx3',
        api_key_primary_val='fzdN623xlQHS7QlgXkg1RtSwOlCl5w1a3MTW5Ul4',
        request_id='017bdf20-82ae-40a9-b5bc-85d07029948a'
    )

    preset_text = [{"role":"system","content":"- 당신은 뉴스기사를 쉽게 풀어주는 텍스트 어시스턴스입니다.\n\n- 주어진 뉴스기사를 분석하여 불필요한 세부사항은 제외하고 핵심내용을 간결하고 명확하게 설명해주세요.\n- 어린이도 이해할 수 있는 난이도로 풀어서 설명하세요.\n- 5문장으로 뉴스를 풀어서 설명하세요.\n- 문장은 ~합니다 체로 이루어져야합니다.\r\n- 마지막 문장은 기사에서 말하고자 하는 증권 또는 주식의 긍정/부정/중립을 글로 풀어내는 내용을 담으세요.\n\n\r- 모든 경제, 증권 전문 용어를 많이 추출하여 설명하세요.\n\n###\n뉴스 풀이: \n문장1. 문장2. 문장3. 문장4. 문장5.\n---\n용어 설명 : \n*용어1\n*용어2\n...\n###\n\n"},
                   {"role":"user","content":"처[\n‘K-컬밸리’ 조성 사업 전면 무산CJ ENM은 CJ라이브시티 지분 90%를 보유 중“CJ ENM, 차입금 대응 무리 없을 것”이 기사는 2024년07월15일 18시02분에 마켓인 프리미엄 콘텐츠로 선공개 되었습니다.[이데일리 마켓in 박미경 기자] ‘세계 최초 K팝 공연 전문 아레나’를 목표로 했던 CJ라이브시티가 8년 만에 백지화됐다. 하지만 모회사인 CJ ENM(035760)의 신용도는 장기적인 관점에서 긍정적이라는 평가가 나온다.\n\n\n\n[그래픽=이데일리 문승용 기자]◇ ‘K-컬처밸리’ 사업 전면 백지화15일 금융투자업계에 따르면 CJ ENM은 이달 초 경기도로부터 ‘K-컬처밸리’ 사업의 계약 해제를 통보받았다. ‘K-컬처밸리’ 조성 사업에는 고양시 일산동구 장항동 일대 경기도 소유 부지에 CJ라이브시티가 총사업비 2조원 가량을 투자해 세계 최대 규모의 K팝 공연장(아레나)과 스튜디오·테마파크·숙박시설·관광단지 등을 조성하는 내용이 담겨 있었다. 경기도는 공공 주도의 공영개발 방식 추진을 발표해 사실상 사업이 전면 무산됐다.이에 따라 모회사인 CJ ENM의 신용도에 미칠 영향에 관심 쏠리고 있다. CJ ENM은 CJ라이브시티 지분 90%를 보유 중이기 때문이다. 현재 한국신용평가와 NICE(나이스)신용평가는 CJ ENM의 신용등급을 ‘AA-(안정적)’로 평가하고 있다.CJ라이브시티는 설립 이후 매년 대규모 적자를 기록한 CJ ENM의 골칫거리 중 하나였다. 지난 2015년 최초 사업계획 수립 이후 6년 만인 지난 2021년 10월 착공에 들어갔으나, 지난해 4월 공사 전면 중단을 선언했다. 원자재 가격, 인건비 상승 등으로 공사비용이 올라간 데다 한국전력공사가 전력 공급 불가를 통보하는 등 대외 환경이 나빠지면서다.지난 2022년 CJ라이브시티는 쌓인 채무보증에 완전 자본잠식 상태에 빠지기도 했다. 지난해 자본잠식에서 벗어난 상황이지만, 현금 및 현금성자산이 2022년 말 115억원에서 2023년 말 33억원으로 급감해 CJ ENM에 연쇄적으로 부담을 주는 상황이었다. “CJ ENM, CJ라이브시티 차입금 대응 문제없어”신용평가사들은 CJ ENM의 신용등급에 즉각적인 영향을 미치진 않을 것이라고 평가했다. CJ라이브시티의 차입금에 대한 대응에 큰 무리가 없다는 판단에서다. 또 장기적으로는 CJ라이브시티와 관련한 리스크가 해소된다는 점에서 투자 부담이 줄어 신용도에 긍정적이라는 분석이다.CJ라이브시티의 차입금은 올해 6월 말 기준 △기업어음(CP) 3000억원 △단기사채 900억원 △CJ ENM으로부터의 차입금 900억원 △토지 유동화 차입금 1400억원 △영구채 600억원 등 총 6800억원에 달하는 것으로 추산됐다. 이중 CP와 단기사채는 CJ ENM이 보증을 섰으며, 영구채의 경우 자금보충약정을 제공했다.토지 유동화 차입금의 경우 토지를 경기도에 반환해 회수하는 대금을 통해 상환할 수 있다는 설명이다. 용지대금에 대한 반환채권을 대주에게 양도하는 양도담보설정계약이 체결돼 있기 때문이다.한신평은 “CJ라이브시티 차입금이 이미 CJ ENM 연결기준 차입금에 포함돼 있다”며 “해당 차입금의 만기 구성과 CJ ENM의 안정적인 사업 기반에 따른 우수한 대외신인도, 보유 비영업용 자산 및 여신한도에 기반한 자금 조달 여력 등을 고려하면 CJ라이브시티 차입금에 대한 대응에는 무리가 없을 것”이라고 밝혔다.NICE신평은 “CJ ENM이 CJ라이브시티 차입금 만기일에 순차적으로 차입금을 상환하면 되므로 단기적으로 유동성 이슈는 제한적”이라며 “CJ라이브시티의 만기 도래 차입금 규모는 2024년 1000억원, 2025년 2000억원, 2026년 이후 1823억원 수준으로 예상된다”고 했다.또 장기적으로 봤을 때 추가적인 자금 소요가 발생하지 않는다는 점에서 신용도에 미치는 영향은 긍정적이라고 판단했다.한 신용평가사 관계자는 “지난 몇 년간 공사 비용이 급격하게 올라 향후 투자 규모가 중요한 모니터링 포인트였다”며 “사업 중단으로 외부 차입 부담이 줄었다는 점이 긍정적”이라고 평가했다.\n"},
]
    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.01,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }

    print(preset_text)
    


[{'role': 'system', 'content': '- 당신은 뉴스기사를 쉽게 풀어주는 텍스트 어시스턴스입니다.\n\n- 주어진 뉴스기사를 분석하여 불필요한 세부사항은 제외하고 핵심내용을 간결하고 명확하게 설명해주세요.\n- 어린이도 이해할 수 있는 난이도로 풀어서 설명하세요.\n- 5문장으로 뉴스를 풀어서 설명하세요.\n- 문장은 ~합니다 체로 이루어져야합니다.\r\n- 마지막 문장은 기사에서 말하고자 하는 증권 또는 주식의 긍정/부정/중립을 글로 풀어내는 내용을 담으세요.\n\n\r- 모든 경제, 증권 전문 용어를 많이 추출하여 설명하세요.\n\n###\n뉴스 풀이: \n문장1. 문장2. 문장3. 문장4. 문장5.\n---\n용어 설명 : \n*용어1\n*용어2\n...\n###\n\n'}, {'role': 'user', 'content': '처[\n‘K-컬밸리’ 조성 사업 전면 무산CJ ENM은 CJ라이브시티 지분 90%를 보유 중“CJ ENM, 차입금 대응 무리 없을 것”이 기사는 2024년07월15일 18시02분에 마켓인 프리미엄 콘텐츠로 선공개 되었습니다.[이데일리 마켓in 박미경 기자] ‘세계 최초 K팝 공연 전문 아레나’를 목표로 했던 CJ라이브시티가 8년 만에 백지화됐다. 하지만 모회사인 CJ ENM(035760)의 신용도는 장기적인 관점에서 긍정적이라는 평가가 나온다.\n\n\n\n[그래픽=이데일리 문승용 기자]◇ ‘K-컬처밸리’ 사업 전면 백지화15일 금융투자업계에 따르면 CJ ENM은 이달 초 경기도로부터 ‘K-컬처밸리’ 사업의 계약 해제를 통보받았다. ‘K-컬처밸리’ 조성 사업에는 고양시 일산동구 장항동 일대 경기도 소유 부지에 CJ라이브시티가 총사업비 2조원 가량을 투자해 세계 최대 규모의 K팝 공연장(아레나)과 스튜디오·테마파크·숙박시설·관광단지 등을 조성하는 내용이 담겨 있었다. 경기도는 공공 주도의 공영개발 방식 추진을 발표해 사실상 사업이 전면 무산됐다.이에 따라 모회사인 CJ ENM의 신용도에 미칠 영향에 관심 쏠리고 있

In [8]:
data = completion_executor.execute(request_data)

In [9]:
print(data)

뉴스 풀이: 
CJ ENM이 8년 동안 추진해 온 'K-컬밸리' 조성 사업이 전면 백지화 되었어요. CJ ENM은 CJ라이브시티 지분 90%를 보유하고 있는데, CJ라이브시티는 매년 대규모 적자를 기록하며 CJ ENM의 골칫거리 중 하나였어요. 이번 사업 무산으로 CJ ENM의 신용도에 영향을 미칠지 관심이 쏠리고 있지만, 신용평가사들은 CJ ENM의 신용등급에 즉각적인 영향을 미치진 않을 것이라고 평가했어요. CJ라이브시티의 차입금에 대한 대응에 큰 무리가 없다는 판단에서이며, 장기적으로는 CJ라이브시티와 관련한 리스크가 해소된다는 점에서 투자 부담이 줄어 신용도에 긍정적이라는 분석이에요.


용어 설명 : 
*아레나: 관객석이 원형으로 배치된 실내 공연장
*자본잠식: 기업의 적자폭이 커져 잉여금이 바닥나고 납입자본금까지 잠식한 상태
*신용등급: 기업의 재무상황 등 여러 요소를 고려하여 신용도를 평가한 등급


In [10]:
df = pd.read_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_최종_cleaned.csv")

In [11]:
df

,news_title,news_url,news_contents,news_dates,cleaned_news_contents
0,줍줍까지 나왔는데 웃돈만 10억…'얼죽신' 열풍에 입주권 귀하신 몸,https://n.news.naver.com/mnews/article/057/000...,[\n\t\t\t【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 ...,2024-07-26 20:06:12,【 앵커멘트 】 '얼죽신'이라는 신조어가 있습니다. 얼어죽어도 신축이라는 새 아파...
1,‘AI 반도체’ 리벨리온 IPO 주관사에 대표 삼성·공동 한투 선정,https://n.news.naver.com/mnews/article/366/000...,[\n기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 ...,2024-07-26 17:46:14,기업가치 5조~7조원 인공지능(AI) 반도체 스타트업 리벨리온이 기업공개(IPO)...
2,서울 세운5-1·3구역 대출 연장 성공…규모 1000억 확대,https://n.news.naver.com/mnews/article/018/000...,[\n대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에...,2024-07-26 19:48:07,대출 2800억 조기 상환…신규 대출 3800억 확대SPC 투모로우세운·에이에이엠...
3,"배임까지… 두산밥캣, 6거래일째 하락 이어 시간外 약세",https://n.news.naver.com/mnews/article/366/000...,[\n\n\n\n\n두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹...,2024-07-26 17:21:13,두산밥캣의 스티어-스키드 로더. /두산밥캣 두산그룹 지배구조 개편 여파로 두산밥캣...
4,상승 출발 엔비디아 vs 하락 출발 테슬라 …과연 누가 웃을까?,https://n.news.naver.com/mnews/article/029/000...,[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자...,2024-07-26 22:56:07,뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자도생...
...,...,...,...,...,...
922,"셀트리온, 하반기 짐펜트라 잠재력 확인-상상인",https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n [파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반...,2024-07-22 05:09:12,[파이낸셜뉴스] 상상인증권은 22일 셀트리온에 대해 올해 하반기는 짐펜트라의 잠재...
923,"열기 빠진 시장…피앤에스미캐닉스 VS 아이빔테크놀로지, 증권가 ‘픽’은",https://n.news.naver.com/mnews/article/018/000...,"[\n[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호...",2024-07-22 00:01:09,"[IPO대IPO]재활로봇 기업, 생체현미경 제조사 이번주 릴레이 청약국내 1호 보..."
924,5대 금융 트래블 체크카드 가입자 650만명 육박,https://n.news.naver.com/mnews/article/001/001...,[\n휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대\n\n\n\n[연합뉴스 ...,2024-07-22 06:15:01,휴가철 앞두고 고객 유치전 치열…신용카드로 경쟁 확대 [연합뉴스 자료사진] (서울...
925,금융당국 채찍 드니까 PF 손실 인식… 대형 증권사도 2분기 어닝쇼크 가능성 커져,https://n.news.naver.com/mnews/article/366/000...,[\n6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기...,2024-07-22 06:01:22,6월부터 PF 사업성 평가 기준 바뀌어빡빡해진 기준에 늘어날 부실 사업장2분기 실...


In [12]:
df['news_contents'][4]

"[\n뉴욕증시 3대 지수 상승출발엔비디아 2%대 상승테슬라는 낙폭 줄여빅테크주 각자도생\n\n\n\n연합뉴스    미국 뉴욕증시 3대 지수가 동반 상승세를 보이고 있다. 개장 전 발표된 미국의 6월 개인소비지출물가(PCE) 상승률이 시장 예상치에 부합하면서 기준금리 인하에 대한 기대감을 한껏 높였다. 빅테크주 가운데 엔비디아는 오르고 있으나 테슬라는 하락 출발했다. 26일(현지시간) 우량주 중심의 다우존스종합지수는 전거래일보다 400포인트 이상 상승 중이다. 대형주 중심의 S&amp;P500지수도 0.7% 대의 오름세를 보이고 있다. 기술주 중심의 나스닥종합지수 역시 0.8%대 상승세를 타고 있다.이틀 연속 큰 폭의 하락세를 보였던 빅테크 기업들의 주가는 엇갈렸다. 전날은 테슬라를 제외하고 모두 급락했으나 이날은 '각자도생'의 주가흐름을 보이고 있다.메타는 2%대 이상 강한 상승세를 타고 있다. 마이크로소프트도 강보합세다. 아마존도 오름세다. 인공지능(AI) 반도체 대장주 엔비디아는 2% 중반대의 비교적 강한 반등세를 보이고 있다. 엔비디아는  강한 변동성을 보이고 있다. 전날에도 장초반 6%대 급락했다가, 장중에 플러스로 반등했다. 그러나 다시 2%대 하락 마감했다.시가총액 3위, 대형주인 엔비디아가 마치 비트코인 등 가상화폐처럼 큰 변동폭을 보임에 따라 전문가들은 투자자들에 신중한 투자를 주문하고 있다. 구글의 모기업 알파벳은 또다시 1% 중반대의 하락세를 보이고 있다. 알파벳은 주가가 24일(현지시간)은 5%대, 25일은 3%대 빠졌다. 'AI 수익성' 논란에 오픈AI가 '서치GPT'를 발표하면서 하락을 부추겼다. 전날 나홀로 1.97%  상승한 전기차 대장주 테슬라는 이날 장 초반 1% 넘는 하락세를 보이고 있다. 이후 낙폭을 줄여가고 있다. 시가총액 1위 애플은 약보합세로 출발했으나 상승 전환했다. ◇금리 인하 연내 3차례?…커지는 기대감 이날 증시 개장에 앞서 미국 상무부는 6월 PCE 가격지수가 작년 동월 대비 2.5% 상승했다고 밝혔다. 전월 

In [13]:
df['summary'] = None

In [14]:
len(df)

927

In [15]:
if __name__ == '__main__':
    completion_executor = CompletionExecutor(
        host='https://clovastudio.stream.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY6lTkg/kCOEv6QZjvkQU61uo3PQoaid6Aols2EIOSYx3',
        api_key_primary_val='fzdN623xlQHS7QlgXkg1RtSwOlCl5w1a3MTW5Ul4',
        request_id='017bdf20-82ae-40a9-b5bc-85d07029948a'
    )
    for i in range(len(df)):
        preset_text = [{"role":"system","content":"- 당신은 뉴스기사를 쉽게 풀어주는 텍스트 어시스턴스입니다.\n\n- 주어진 뉴스기사를 분석하여 불필요한 세부사항은 제외하고 핵심내용을 간결하고 명확하게 설명해주세요.\n- 어린이도 이해할 수 있는 난이도로 풀어서 설명하세요.\n- 5문장으로 뉴스를 풀어서 설명하세요.\n- 문장은 ~합니다 체로 이루어져야합니다.\r\n- 마지막 문장은 기사에서 말하고자 하는 증권 또는 주식의 긍정/부정/중립을 글로 풀어내는 내용을 담으세요.\n\n\r- 모든 경제, 증권 전문 용어를 많이 추출하여 설명하세요.\n\n###\n뉴스 풀이: \n문장1. 문장2. 문장3. 문장4. 문장5.\n---\n용어 설명 : \n*용어1\n*용어2\n...\n###\n\n"},
                       {"role":"user","content":df.loc[i, 'news_contents']},
    ]
        request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.01,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 1
    }
        summary_content = completion_executor.execute(request_data)
        df.loc[i, 'summary'] = summary_content
        print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
df.to_csv("C:/동국대학교/공모전/미래에셋/뉴스데이터_최종_cleaned_요약.csv", encoding='utf-8', index=False)

In [21]:
df['summary'][26]

'뉴스 풀이 : \n도널드 트럼프 전 미국 대통령이 피격 사건 이후에도 건재한 모습을 보이자, 국내 증시에서 방위산업 관련주가 상승했습니다. LIG넥스원과 현대로템은 52주 최고가를 경신했고, 한화시스템, 한화에어로스페이스, 한국항공우주 등도 동반 상승했습니다. 이는 트럼프 전 대통령의 당선 가능성이 높아졌다는 관측에 따른 것으로, 트럼프 전 대통령은 자국 우선주의를 강조하는 인물로 방위비를 크게 늘릴 것으로 예상되어 국내 방산 기업들의 수출 물량이 늘어날 것으로 기대되기 때문입니다.\n\n용어 설명 : \n*방위산업 관련주 : 국가의 안보와 관련된 방위산업과 연관된 기업의 주식을 말합니다.\n*52주 최고가 : 주식 시장에서 해당 종목의 최근 1년간 가장 높은 주가를 말합니다.\n*자국 우선주의 : 다른 나라와의 협력보다는 자신의 국가의 이익을 우선시하는 정책을 말합니다.\n\n---\n\n방위산업 관련 주식들이 상승하고 있습니다. 이는 트럼프 전 미국 대통령의 피격 사건 이후, 그의 당선 가능성이 높아졌기 때문입니다. 트럼프 전 대통령은 자주 국방 기조를 강조하며, 방위비를 늘릴 것으로 예상되어 국내 방산 기업들의 수출 물량이 늘어날 것이라는 기대감이 반영되었습니다. LIG넥스원, 현대로템 등 주요 방산 관련주들이 52주 최고가를 경신하는 등 상승세를 보이고 있습니다. 다만, 전문가들은 이번 피습 사건과 같은 정치적 이슈는 단기적인 영향에 그칠 수 있으며, 중장기적으로는 실적에 따라 주가가 움직일 것으로 전망하고 있습니다.'

In [30]:
print(df['summary'][17])

뉴스 풀이: 
최근 미국의 금리 인하를 기대하는 분위기가 커지고 있습니다. 이는 미국의 고용지표와 소비자물가지수(CPI)가 발표된 후 더욱 가속화되고 있습니다. 샴의 법칙은 실업률의 3개월 이동평균치가 지난 12개월 최저 실업률보다 0.5%포인트 높아지면 경제가 침체 상태로 들어간다는 지표로, 최근 이 지표가 상승하면서 주목받고 있습니다.

하지만, 현재 미국 경기는 침체라고 보기에는 너무 뜨겁습니다. 2022년 상반기 미국의 장단기 금리가 역전된 국면에서도 미국 경제는 역성장했지만, 지금은 금리 인하 시점에 대한 기대가 계속 미뤄지고 있습니다. 이는 팬데믹이 발생시킨 특수한 상황들에서 원인을 찾을 수 있습니다.

경기침체에 대한 우려가 올해 하반기 금리 인하 결정에 큰 영향을 미치지 않을 가능성이 큽니다. 올해 성장률이 잠재성장률보다 높다면 1~2회 정도 인하를 예상하며, 지난해 성장률 2.5%에서 크게 차이가 나지 않는다면 올해 금리 인하는 한번 정도에 그칠 가능성이 높습니다.

용어 설명 : 
*샴의 법칙 : 실업률의 3개월 이동평균치가 지난 12개월 최저 실업률보다 0.5%포인트 높아지면 경제가 침체 상태로 들어간다는 지표
*크레디트 스프레드 : 신용위험지표 중 하나로, 채권 발행 시 신용등급이 낮은 채권의 금리와 신용등급이 높은 채권의 금리 차이를 의미함
